#### ***Model build***

##### ***Aim:*** 
- To show the seasonality of the annual plastic pollution, predicted from the precipitation data

##### ***Context:***
- Building on The Ocean Cleanup model which shows an annual pollution level for 1000 rivers
- Focusing on the top 10 polluting rivers in South East Asia
- Aiming to vizualise this on a map, with an interactive slider across each month
- In my simplified version, precipitation is used as a proxy for Mobilization (P(M)) from The Ocean Cleanup model
    - Mobilization is then related to plastic pollution


